In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, svm
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import DecisionBoundaryDisplay

In [ ]:
# 1. Preparação dos Dados
# Usamos 'make_moons' com um pouco mais de ruído para tornar a tarefa desafiadora
X, y = datasets.make_moons(n_samples=300, noise=0.25, random_state=42)

In [ ]:
# Dividir em treino e teste para avaliação justa (ROC deve ser calculado no teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
print(X)

In [ ]:
# Configuração dos 3 Modelos (Os Especialistas)
# Nota: probability=True é necessário se quiséssemos usar predict_proba,
# mas para ROC em SVM, usar decision_function é mais direto e eficiente.
models = [
    svm.SVC(kernel='linear', C=1.0),
    svm.SVC(kernel='poly', degree=3, C=1.0),
    svm.SVC(kernel='rbf', gamma=0.7, C=1.0)
]

titles = ["Kernel Linear", "Kernel Polinomial (Grau 3)", "Kernel RBF"]

In [ ]:
# Configuração da Figura (2 linhas: Fronteiras e ROC)
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
plt.subplots_adjust(hspace=0.3)

for i, model in enumerate(models):
    # --- Treinamento ---
    model.fit(X_train, y_train)

    # --- Plot 1: Fronteira de Decisão (Linha Superior) ---
    ax_bound = axes[0, i]
    DecisionBoundaryDisplay.from_estimator(
        model, X_train,
        cmap=plt.cm.coolwarm,
        plot_method="pcolormesh",
        shading="auto",
        alpha=0.4,
        ax=ax_bound
    )
    ax_bound.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=plt.cm.coolwarm, edgecolors='k', s=20)
    ax_bound.set_title(f"Fronteira: {titles[i]}", fontsize=11, fontweight='bold')
    ax_bound.set_xticks(())
    ax_bound.set_yticks(())

    # --- Plot 2: Curva ROC (Linha Inferior) ---
    ax_roc = axes[1, i]

    # Para SVM, usamos decision_function para obter o "score" (distância da margem)
    y_score = model.decision_function(X_test)

    # Cálculo da ROC e AUC
    fpr, tpr, _ = roc_curve(y_test, y_score)
    roc_auc = auc(fpr, tpr)

    # Plotagem da curva
    ax_roc.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.2f}')
    ax_roc.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--') # Linha do acaso
    ax_roc.set_xlim([0.0, 1.0])
    ax_roc.set_ylim([0.0, 1.05])
    ax_roc.set_xlabel('Taxa de Falsos Positivos (FPR)')
    ax_roc.set_ylabel('Taxa de Verdadeiros Positivos (TPR)')
    ax_roc.set_title(f'ROC: {titles[i]}')
    ax_roc.legend(loc="lower right")
    ax_roc.grid(True, alpha=0.3)

In [ ]:
plt.suptitle("Comparação de Kernels SVM: Visão Geométrica e Performance (ROC)", fontsize=16)
plt.show()

# Feedback textual
print("--- ANÁLISE DAS MÉTRICAS ---")
print("Observe a métrica AUC (Area Under Curve) na legenda dos gráficos inferiores.")
print("1. AUC = 0.5: O modelo é um 'chute' aleatório (linha tracejada).")
print("2. AUC = 1.0: O modelo é perfeito.")
print("Compare como o Kernel RBF geralmente obtém a maior AUC neste dataset curvo.")